In [1]:
import pandas as pd
# import swifter
import glob

from pandarallel import pandarallel

In [2]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
trip_df = pd.read_csv('cleaned_trips.csv')
trip_df['Start Time'], trip_df['End Time'] = pd.to_datetime(trip_df['Start Time']), pd.to_datetime(trip_df['End Time'])
trip_df.head()

,Unnamed: 0,Trip Id,Start Time,End Time,Trip Duration,Start Station Id,Start Station Name,End Station Id,End Station Name,User Type
0,0,712382,2017-01-01 00:00:00,2017-01-01 00:03:00,223,7051,Mutual St / Shuter St,7089,Church St / Wood St,Member
1,1,712383,2017-01-01 00:00:00,2017-01-01 00:05:00,279,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member
2,2,712384,2017-01-01 00:05:00,2017-01-01 00:29:00,1394,7113,Parliament St / Aberdeen Ave,7199,College St / Markham St,Member
3,3,712385,2017-01-01 00:07:00,2017-01-01 00:21:00,826,7077,College Park South,7010,King St W / Spadina Ave,Member
4,4,712386,2017-01-01 00:08:00,2017-01-01 00:12:00,279,7079,McGill St / Church St,7047,University Ave / Gerrard St W (East Side),Member


In [4]:
temp_df = pd.read_csv('Final/weatherstats_toronto_hourly.csv')[['date_time_local', 'temperature']].sort_values('date_time_local').fillna(method='ffill').reset_index(drop=True)

/tmp/ipykernel_1245268/1754863073.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df = pd.read_csv('Final/weatherstats_toronto_hourly.csv')[['date_time_local', 'temperature']].sort_values('date_time_local').fillna(method='ffill').reset_index(drop=True)


In [5]:
temp_df['date_time_local'] = pd.to_datetime(temp_df['date_time_local'].str.replace('EST|EDT', '', regex=True))

In [6]:
temp_df = temp_df.groupby('date_time_local').mean().reset_index() # deals with multiple recorded temps in the raw data, caused by EST, EDT changes for example

In [12]:
from datetime import datetime, timedelta

start_date = min(trip_df['Start Time'].min().floor('d'), trip_df['End Time'].min().floor('d'))
end_date = max(trip_df['Start Time'].max().ceil('d'), trip_df['End Time'].max().ceil('d'))

temp_dates = [start_date + timedelta(hours=i) for i in range(int((end_date - start_date).total_seconds() // 3600) + 1)]

new_temp_df = pd.DataFrame(data={'date_time_local': temp_dates}, dtype='datetime64[ns]')

In [14]:
new_temp_df = pd.merge(new_temp_df, temp_df, how='left').groupby('date_time_local').mean().reset_index() # deals with nan values resulted from left join

In [16]:
# new_temp_df['temperature'] = new_temp_df['temperature'].fillna(method='ffill')
new_temp_df['temperature'] = new_temp_df['temperature'].ffill()

In [18]:
from datetime import datetime, timedelta

def avg_temp(tstart, tstop):

    assert tstart < tstop

    total = 0
    minutes = 0
    
    tc = tstart
    
    while tc < tstop:

        try:
    
            tc_floor = tc.floor('h')
        
            next_t = min(tstop, tc_floor + timedelta(hours=1))
        
            minutes_in_interval = (next_t - tc).total_seconds() // 60
        
            current_temp = new_temp_df[new_temp_df['date_time_local'] == tc_floor].iloc[0, 1]
        
            minutes += minutes_in_interval
        
            total += (minutes_in_interval * current_temp)
        
            tc = next_t

        except Exception as E:
            print(tc)
            print(tc_floor)

    return total / minutes


def avg_temp2(tstart, tstop): # Should change resolution to seconds since there are trips under a minute

    assert tstart < tstop

    total = 0
    minutes = 0
    
    tc = tstart
    temp_index = new_temp_df[new_temp_df['date_time_local'] == tstart.floor('h')].index.to_numpy()[0]
    
    while tc < tstop:

        # try:
    
        tc_floor = tc.floor('h')
    
        next_t = min(tstop, tc_floor + timedelta(hours=1))
    
        minutes_in_interval = (next_t - tc).total_seconds() // 60
    
        current_temp = new_temp_df.iloc[temp_index, 1]

        temp_index += 1
    
        minutes += minutes_in_interval
    
        total += (minutes_in_interval * current_temp)
    
        tc = next_t

        # except Exception as E:
        #     print(tc)
        #     print(tc_floor)

    return total / minutes


def avg_temp3(tstart, tstop): # Should change resolution to seconds since there are trips under a minute

    assert tstart < tstop

    total = 0
    seconds = 0
    
    tc = tstart
    temp_index = new_temp_df[new_temp_df['date_time_local'] == tstart.floor('h')].index.to_numpy()[0]
    
    while tc < tstop:

        # try:
    
        tc_floor = tc.floor('h')
    
        next_t = min(tstop, tc_floor + timedelta(hours=1))
    
        seconds_in_interval = (next_t - tc).total_seconds()
    
        current_temp = new_temp_df.iloc[temp_index, 1]

        temp_index += 1
    
        seconds += seconds_in_interval
    
        total += (seconds_in_interval * current_temp)
    
        tc = next_t

        # except Exception as E:
        #     print(tc)
        #     print(tc_floor)

    return total / seconds

In [176]:
import time

start = time.time()
trip_df['temperature'] = trip_df.apply(lambda x: avg_temp(x['Start Time'], x['End Time']), axis=1)
end = time.time()

print(f"{(end - start):0.2f}")

1647.79


In [212]:
import time

start = time.time()
trip_df['temperature'] = trip_df.apply(lambda x: avg_temp2(x['Start Time'], x['End Time']), axis=1)
end = time.time()

print(f"{(end - start):0.2f}")

1147.26


In [19]:
import time

start = time.time()
trip_df['temperature'] = trip_df.parallel_apply(lambda x: avg_temp3(x['Start Time'], x['End Time']), axis=1)
end = time.time()

print(f"{(end - start):0.2f}")

677.64


In [20]:
trip_df = trip_df.drop('Unnamed: 0', axis=1)
trip_df

,Trip Id,Start Time,End Time,Trip Duration,Start Station Id,Start Station Name,End Station Id,End Station Name,User Type,temperature
0,712382,2017-01-01 00:00:00,2017-01-01 00:03:00,223,7051,Mutual St / Shuter St,7089,Church St / Wood St,Member,0.500000
1,712383,2017-01-01 00:00:00,2017-01-01 00:05:00,279,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member,0.500000
2,712384,2017-01-01 00:05:00,2017-01-01 00:29:00,1394,7113,Parliament St / Aberdeen Ave,7199,College St / Markham St,Member,0.500000
3,712385,2017-01-01 00:07:00,2017-01-01 00:21:00,826,7077,College Park South,7010,King St W / Spadina Ave,Member,0.500000
4,712386,2017-01-01 00:08:00,2017-01-01 00:12:00,279,7079,McGill St / Church St,7047,University Ave / Gerrard St W (East Side),Member,0.500000
...,...,...,...,...,...,...,...,...,...,...
25242403,2383640,2017-12-31 23:37:43,2017-12-31 23:43:00,317,7152,Ossington Ave / Bloor St W,7184,Ossington Ave / College St,Member,-17.700000
25242404,2383641,2017-12-31 23:42:06,2017-12-31 23:58:24,978,7012,Elizabeth St / Edward St (Bus Terminal),7184,Ossington Ave / College St,Member,-17.700000
25242405,2383643,2017-12-31 23:47:13,2018-01-01 00:11:40,1467,7061,Dalton Rd / Bloor St W,7175,HTO Park (Queens Quay W),Casual,-18.558896
25242406,2383644,2017-12-31 23:47:40,2017-12-31 23:57:49,609,7132,Kendal Ave / Spadina Rd,7189,Augusta Ave / Denison Sq,Member,-17.700000


In [ ]:
trip_df.to_csv('data_temperature_augmented.csv')

In [48]:
import numpy as np
edge_count=6
edges = np.linspace(trip_df['temperature'].min(), trip_df['temperature'].max(), edge_count)
edges

array([-23.  , -11.44,   0.12,  11.68,  23.24,  34.8 ])

In [49]:
def apply_temp_groups(temp, min_t=-23, max_t=34.8, interval_count=5):
        
    interval_length = (max_t - min_t) / (interval_count)

    clipped = min((temp - min_t) // interval_length, interval_count - 1)

    if clipped is None:
        print(temp)
    
    return int(clipped)

In [50]:
trip_df['group_temp'] = trip_df['temperature'].apply(apply_temp_groups)

In [51]:
trip_df

,Trip Id,Start Time,End Time,Trip Duration,Start Station Id,Start Station Name,End Station Id,End Station Name,User Type,temperature,group_temp
0,712382,2017-01-01 00:00:00,2017-01-01 00:03:00,223,7051,Mutual St / Shuter St,7089,Church St / Wood St,Member,0.500000,2
1,712383,2017-01-01 00:00:00,2017-01-01 00:05:00,279,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member,0.500000,2
2,712384,2017-01-01 00:05:00,2017-01-01 00:29:00,1394,7113,Parliament St / Aberdeen Ave,7199,College St / Markham St,Member,0.500000,2
3,712385,2017-01-01 00:07:00,2017-01-01 00:21:00,826,7077,College Park South,7010,King St W / Spadina Ave,Member,0.500000,2
4,712386,2017-01-01 00:08:00,2017-01-01 00:12:00,279,7079,McGill St / Church St,7047,University Ave / Gerrard St W (East Side),Member,0.500000,2
...,...,...,...,...,...,...,...,...,...,...,...
25242403,2383640,2017-12-31 23:37:43,2017-12-31 23:43:00,317,7152,Ossington Ave / Bloor St W,7184,Ossington Ave / College St,Member,-17.700000,0
25242404,2383641,2017-12-31 23:42:06,2017-12-31 23:58:24,978,7012,Elizabeth St / Edward St (Bus Terminal),7184,Ossington Ave / College St,Member,-17.700000,0
25242405,2383643,2017-12-31 23:47:13,2018-01-01 00:11:40,1467,7061,Dalton Rd / Bloor St W,7175,HTO Park (Queens Quay W),Casual,-18.558896,0
25242406,2383644,2017-12-31 23:47:40,2017-12-31 23:57:49,609,7132,Kendal Ave / Spadina Rd,7189,Augusta Ave / Denison Sq,Member,-17.700000,0


In [54]:
## Temperature data
for group in trip_df['group_temp'].unique():
    print(group)
    group_df = trip_df[trip_df['group_temp'] == group]
    group_df.to_csv(f'bike_share_toronto_agumented_group{group}.csv')

2
1
3
4
0


#### one function to do it all

In [ ]:
from threshold.similarity import compute_avg_similarity
import random

def get_tempdf(df):
    temp1_df = df[["Start Station Id", "Start Station Name"]].rename({'Start Station Id':'Id', 'Start Station Name': 'Name'}, axis=1)
    temp2_df = df[["End Station Id", "End Station Name"]].rename({'End Station Id':'Id', 'End Station Name': 'Name'}, axis=1)
    temp_df = pd.concat([temp1_df, temp2_df]).drop_duplicates()
    return temp_df

def change_membership(membership):
    if membership == 'Annual Member':
        return 'Member'
    elif membership == 'Casual Member':
        return 'Casual'
    else:
        return membership
        
def func1(df):
    temp_df = get_tempdf(df)
    temp_temp_df = (temp_df.groupby("Id").count() > 1).reset_index()


    threshold = 0.33
    changes = []
    
    # we are trying to find the mapping and after we've found one, we replace the duplicates
    for id in temp_temp_df[temp_temp_df['Name'] == True].Id.to_numpy():
        duplicates = temp_df[temp_df['Id'] == id]
        size = len(duplicates)
    
        similarity = compute_avg_similarity(duplicates.Name.to_numpy())
    
        if similarity > threshold:
            choice = size - 1
        else:
            choice = size
        
        # if any of the choices -> add a tuple for every other choice
        # else do nothing
        if choice < size:
            
            choice_name = duplicates.iloc[choice, 1]
    
            for i in range(size):
                if i == choice:
                    continue
                changes.append((duplicates.iloc[i, 1], choice_name))


    for change in changes:
        
        df.loc[df['Start Station Name'] == change[0], 'Start Station Name'] = change[1]
    
        df.loc[df['End Station Name'] == change[0], 'End Station Name'] = change[1]

    temp_df = get_tempdf(df)
    new_changes = []

    for id in [7117, 7130, 7291, 7381]:
        duplicates = temp_df[temp_df['Id'] == id]
        size = len(duplicates)

        if size == 0:
            continue
    
        choice = size - 1
            
        choice_name = duplicates.iloc[choice, 1]
    
        for i in range(size):
            if i == choice:
                continue
            new_changes.append((duplicates.iloc[i, 1], choice_name))


    comb_to_change = [((7001, "Lower Jarvis St / The Esplanade"), 7686)]

    for change in comb_to_change:
    
        df.loc[(df['Start Station Id'] == change[0][0]) & (df['Start Station Name'] == change[0][1]), 'Start Station Id'] = change[1]
    
        df.loc[(df['End Station Id'] == change[0][0]) & (df['End Station Name'] == change[0][1]), 'End Station Id'] = change[1]

    id_to_change = [(7720, 7372), (7685, 7322), (7712, 7072), (7680, 7510), (7792, 7098), (7470, 7398)]
    
    for change in id_to_change:
    
        df.loc[df['Start Station Id'] == change[0], 'Start Station Id'] = change[1]
    
        df.loc[df['End Station Id'] == change[0], 'End Station Id'] = change[1]

    available_ids = list(set(range(7000,8000)) - set(temp_df['Id'].unique()))

    new_id_changes = []
    
    temp_temp_df = (temp_df.groupby("Id").count() > 1).reset_index()
    
    for id in temp_temp_df[temp_temp_df['Name'] == True].Id.to_numpy():
    
        # choose one of the options to change id for
        duplicates = temp_df[temp_df['Id'] == id]
        size = len(duplicates)
    
        choice = size - 1 
        choice_name = duplicates.iloc[choice, 1]
    
        new_id = random.choice(available_ids)
        available_ids.remove(new_id)
    
        new_id_changes.append(((id, choice_name), new_id))

    for id_change in new_id_changes:
    
        df.loc[(df['Start Station Id'] == id_change[0][0]) & (df['Start Station Name'] == id_change[0][1]), 'Start Station Id'] = id_change[1]
    
        df.loc[(df['End Station Id'] == id_change[0][0]) & (df['End Station Name'] == id_change[0][1]), 'End Station Id'] = id_change[1]

    df['User Type'] = df['User Type'].apply(change_membership)

    df = df[df['Trip Duration'] > 29]

    df = df[df['Start Station Id'] != df['End Station Id']]

    df = df[df['Start Time'] < df['End Time']]

    return df

In [9]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import networkx as nx

def func2(df):
    
    whole_trips_df = df[['Start Time', 'End Time', 'Trip Id', 'Trip Duration', 'Start Station Id', 'End Station Id']]
    whole_trips_df = whole_trips_df.astype({'Start Time': 'datetime64[ns]', 'End Time': 'datetime64[ns]'})

    temp_df = whole_trips_df[['Start Time', 'End Time', 'Start Station Id', 'End Station Id', 'Trip Id']].groupby(['Start Station Id', 'End Station Id']).agg({'Trip Id': 'count', 'Start Time': 'min', 'End Time': 'max'}).reset_index().rename({'Trip Id': 'Trip Count'}, axis=1)
    temp_df['Link Duration'] = temp_df.apply(lambda x: (x['End Time'] - x['Start Time']).ceil('d'), axis=1).dt.days    #.astype('timedelta64[D]')
    temp_df['Avg Duration'] = whole_trips_df[['Start Station Id', 'End Station Id', 'Trip Duration']].groupby(['Start Station Id', 'End Station Id']).mean().reset_index()['Trip Duration'] # why using trip duration and not link duration
    temp_df['duration_weights'] = temp_df['Avg Duration'].apply(lambda x: 1 / x)

    temp_df['tpd_weights'] = temp_df.apply(lambda x: x['Trip Count'] / x['Link Duration'], axis=1)
    temp_df = temp_df.rename({'Trip Count': 'trip_count_weights'}, axis=1).drop(['Start Time', 'End Time', 'Link Duration', 'Avg Duration'], axis=1)

    nnp = np.percentile(temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']], 99, axis=0)
    temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']] = np.clip(temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']], None, nnp)

    min_max_scaler = MinMaxScaler((1, 10))
    x_scaled = min_max_scaler.fit_transform(temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']])
    temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']] = x_scaled

    G = nx.from_pandas_edgelist(temp_df, source="Start Station Id", target="End Station Id", edge_attr=['trip_count_weights', 'duration_weights', 'tpd_weights'], create_using=nx.DiGraph)

    node_labels = {d['Id']: d['Name'] for d in get_tempdf(df)[['Id', 'Name']].to_dict('records')}
    nx.set_node_attributes(G, node_labels, name='Name')

    return G, temp_df

In [10]:
import matplotlib.pyplot as plt

def func3(G, name, deg_ext=None, bins=9):

    # print(f"{name}:")
    nodes, edges = G.number_of_nodes(), G.number_of_edges()

    in_degrees = [deg for (id, deg) in G.in_degree() if deg > 0]
    out_degrees = [deg for (id, deg) in G.out_degree() if deg > 0]

    if deg_ext:
        in_kmin, in_kmax, out_kmin, out_kmax = deg_ext
    else:
        in_kmin = min(in_degrees)
        in_kmax = max(in_degrees)
        out_kmin = min(out_degrees)
        out_kmax = max(out_degrees)

    # Get 10 logarithmically spaced bins between kmin and kmax
    # in_bin_edges_log = np.logspace(np.log10(in_kmin + 1e-1), np.log10(in_kmax), num=bins + 1)
    # out_bin_edges_log = np.logspace(np.log10(out_kmin + 1e-1), np.log10(out_kmax), num=bins + 1)
    in_bin_edges_log = np.logspace(np.log10(in_kmin), np.log10(in_kmax), num=bins + 1)
    out_bin_edges_log = np.logspace(np.log10(out_kmin), np.log10(out_kmax), num=bins + 1)

    # histogram the data into these bins
    in_density_log, _ = np.histogram(in_degrees, bins=in_bin_edges_log, density=True)    
    out_density_log, _ = np.histogram(out_degrees, bins=out_bin_edges_log, density=True)
    
    # "x" should be midpoint (IN LOG SPACE) of each bin
    in_log_be_log = np.log10(in_bin_edges_log)
    in_x_log = 10**((in_log_be_log[1:] + in_log_be_log[:-1])/2)
    
    out_log_be_log = np.log10(out_bin_edges_log)
    out_x_log = 10**((out_log_be_log[1:] + out_log_be_log[:-1])/2)

    # Get 20 logarithmically spaced bins between kmin and kmax
    in_bin_edges = np.linspace(in_kmin, in_kmax, num=bins + 1)
    out_bin_edges = np.linspace(out_kmin, out_kmax, num=bins + 1)
    
    # histogram the data into these bins
    in_density, _ = np.histogram(in_degrees, bins=in_bin_edges, density=True)
    out_density, _ = np.histogram(out_degrees, bins=out_bin_edges, density=True)

    # "x" should be midpoint (IN LOG SPACE) of each bin
    in_log_be = np.log10(in_bin_edges)
    in_x = 10**((in_log_be[1:] + in_log_be[:-1])/2)
    
    out_log_be = np.log10(out_bin_edges)
    out_x = 10**((out_log_be[1:] + out_log_be[:-1])/2)

    fig, axis = plt.subplots(2, 2, figsize=(12,9))

    fig.suptitle(f"{name}: nodes = {nodes}, edges = {edges}")

    axis = axis.ravel()

    axis[0].loglog(in_x_log, in_density_log, marker='o', linestyle='none')
    axis[0].set_xlabel(r"in degree $k$ - loglog", fontsize=16)
    axis[0].set_ylabel(r"$P(k)$", fontsize=16)
    
    axis[1].loglog(out_x_log, out_density_log, marker='o', linestyle='none')
    axis[1].set_xlabel(r"out degree $k$ - loglog", fontsize=16)
    axis[1].set_ylabel(r"$P(k)$", fontsize=16)
    
    axis[2].plot(in_x, in_density, marker='o', linestyle='none')
    axis[2].set_xlabel(r"in degree $k$", fontsize=16)
    axis[2].set_ylabel(r"$P(k)$", fontsize=16)
    
    axis[3].plot(out_x, out_density, marker='o', linestyle='none')
    axis[3].set_xlabel(r"out degree $k$", fontsize=16)
    axis[3].set_ylabel(r"$P(k)$", fontsize=16)
    
    # save the plot
    plt.tight_layout()
    # plt.show()
    fig.savefig(f"figs3/{name.split('.')[0]}.png")
    plt.close()
